In [2]:
# https://colab.research.google.com/github/tensorflow/tflite-micro/blob/main/tensorflow/lite/micro/examples/hello_world/train/train_hello_world_model.ipynb#scrollTo=aCZBFzjClURz

import os
import sys
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt

print("Tensorflow version " + tf.__version__)
if tf.__version__ != '2.5.0': 
    print('Needs to run on tf 2.5.0!')

Tensorflow version 2.5.0


In [3]:
df = pd.read_json('../data/labelled_data/prepared-samples-binary.json')

y = df['label'].to_numpy()
X = np.array(df['data'].to_list())
X = np.expand_dims(X, axis=3)
print(y.shape)
print(X.shape)


(200,)
(200, 32, 32, 1)


In [4]:
# Create train / val / test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [5]:
# Set seed for experiment reproducibility
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)


In [8]:
# Define paths to model files
MODELS_DIR = 'models/'
if not os.path.exists(MODELS_DIR):
    os.mkdir(MODELS_DIR)
MODEL_TF = MODELS_DIR + 'squeezenet-binary'
MODEL_NO_QUANT_TFLITE = MODELS_DIR + 'squeezenet-binary_no_quant.tflite'
MODEL_TFLITE = MODELS_DIR + 'squeezenet-binary.tflite'


In [9]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_TF)
model_no_quant_tflite = converter.convert()


In [10]:

# Save the model to disk
open(MODEL_NO_QUANT_TFLITE, "wb").write(model_no_quant_tflite)


494608

In [12]:

## Convert the model to the TensorFlow Lite format with quantization
def representative_dataset():
  for i in range(50):
    # for CNN the input is NOT flattened
    yield([X_train[i].reshape(1, 32, 32, 1).astype(np.float32)])
# Set the optimization flag.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Enforce integer only quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
# Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()


In [13]:

# Save the model to disk
open(MODEL_TFLITE, "wb").write(model_tflite)


160696

In [14]:

# Calculate size
# careful - this does not count everything properly
# size_tf = os.path.getsize(MODEL_TF)
# better version:
from pathlib import Path
root_directory = Path(MODEL_TF)
size_tf = sum(f.stat().st_size for f in root_directory.glob('**/*') if f.is_file())

size_no_quant_tflite = os.path.getsize(MODEL_NO_QUANT_TFLITE)
size_tflite = os.path.getsize(MODEL_TFLITE)


In [15]:

# Compare size
size_comparison = pd.DataFrame.from_records(
  [["TensorFlow", f"{size_tf} bytes", ""],
    ["TensorFlow Lite", f"{size_no_quant_tflite} bytes ", f"(reduced by {size_tf - size_no_quant_tflite} bytes)"],
    ["TensorFlow Lite Quantized", f"{size_tflite} bytes", f"(reduced by {size_no_quant_tflite - size_tflite} bytes)"]],
    columns = ["Model", "Size", ""], index="Model")

print(size_comparison)


                                    Size                            
Model                                                               
TensorFlow                 2996689 bytes                            
TensorFlow Lite            494608 bytes   (reduced by 2502081 bytes)
TensorFlow Lite Quantized   160696 bytes   (reduced by 333912 bytes)
